In [1]:
import json

In [16]:
!ls ../multihop_dense_retrieval/data/hotpot/hotpot_qas_val.json

dev_retrieval_b50_k50_sp.json  hotpot_test_fullwiki_v1.json
dev_retrieval_top100_sp.json   hotpot_train_firsthop.json
hotpot_dev_fullwiki_v1.json    hotpot_train_v1.1.json
hotpot_dev_with_neg_v0.json    hotpot_train_with_neg_v0.json
hotpot_qas_val.json	       train_retrieval_b100_k100_sp.json


In [2]:
SPLIT = "train"

In [7]:
# Read training data
filepaths = {
    'train': {
        'original': '../multihop_dense_retrieval/data/hotpot/hotpot_train_v1.1.json',
        'mdr': '../multihop_dense_retrieval/data/hotpot/hotpot_train_with_neg_v0.json'
    },
    'dev': {
        'original': '../multihop_dense_retrieval/data/hotpot/hotpot_dev_fullwiki_v1.json',
        'mdr': '../multihop_dense_retrieval/data/hotpot/hotpot_dev_with_neg_v0.json'
    }
}


fp_hotpot_mdr = filepaths[SPLIT]['mdr']
fp_hotpot_original = filepaths[SPLIT]['original']

hotpot_mdr = []
with open(fp_hotpot_mdr, 'r') as handle:
    lines = handle.readlines()
    for l in lines:
        hotpot_mdr.append(json.loads(l))
    
with open(fp_hotpot_original, 'r') as handle:
    hotpot_orig = json.load(handle)

In [8]:
# Training data length
len(hotpot_orig)

7405

In [9]:
# Example training object
hotpot_orig[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Adam Collis',
   ['Adam Collis is an American filmmaker and actor.',
    ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
    ' He also studied cinema at the University of Southern California from 1991 to 1997.',
    ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
    ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']],
  ['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lug

In [13]:
# Count number of questions for which the context is not available for the SUP sentences

count_one, count_both, not_two = 0,0,0
for h in hotpot_orig:
    sup_set = set()
    for sup in h['supporting_facts']:
        sup_set.add(sup[0])
    context_set = set()
    for context in h['context']:
        context_set.add(context[0])
    if len(sup_set.intersection(context_set)) == 0:
        print(h)
        break
        count_both += 1
    if len(sup_set) <2:
        not_two += 1
print(count_both)
print(not_two)

{'_id': '5a8b57f25542995d1e6f1371', 'answer': 'yes', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]], 'context': [['Adam Collis', ['Adam Collis is an American filmmaker and actor.', ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.', ' He also studied cinema at the University of Southern California from 1991 to 1997.', ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).', ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']], ['Ed Wood (film)', ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.', " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.", 

In [11]:
# Check if the MDR-processed hotpot data is the same length
len(hotpot_mdr)

7405

In [14]:
# Example object in MDR hotpot
hotpot_mdr[0]

{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answers': ['yes'],
 'type': 'comparison',
 'pos_paras': [{'title': 'Ed Wood',
   'text': 'Edward Davis Wood Jr. (October 10, 1924\xa0– December 10, 1978) was an American filmmaker, actor, writer, producer, and director.'},
  {'title': 'Scott Derrickson',
   'text': 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. He lives in Los Angeles, California. He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."'}],
 'neg_paras': [{'title': 'That Darn Cat (1997 film)',
   'text': 'That Darn Cat is a 1997 American mystery comedy film starring Christina Ricci and Doug E. Doug. It is a remake of the 1965 film "That Darn Cat!", which in turn was based on the book "Undercover Cat" by Gordon and Mildred Gordon. It is directed by British TV ve

In [29]:
def extract_sentences(sup_idx_list, context_list, doc_title=None):
    return_obj = []
    sup = {}
    for s in sup_idx_list:
        if s[0] not in sup:
            sup[s[0]] = []
        sup[s[0]].append(s[1])
    for c in context_list:
        if c[0] not in sup:
            continue
        if doc_title is not None and c[0] != doc_title:
            continue
        for i in sup[c[0]]:
            try:
                return_obj.append(c[1][i])
            except:
                print("IndexError: ignoring")
                print(c[0])
                print(c[1])
                print(sup[c[0]])
    return return_obj

In [30]:
output = []

for i in range(len(hotpot_orig)):
    orig_obj = hotpot_orig[i]
    mdr_obj = hotpot_mdr[i]
    q_obj = {
        "id": orig_obj["_id"],
        "question": orig_obj["question"],
        "level": orig_obj["level"],
        "type": orig_obj["type"],
        "answers": [],
        "final_answer": orig_obj["answer"],
    }
    if orig_obj["type"] == "comparison":
        # Fetch all SUP sentences into "answers"
        q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], orig_obj["context"])
    else:
        # Fetch only the first-hop SUP sentences into "answers"
        for sup in orig_obj["supporting_facts"]:
            if sup[0] != mdr_obj["bridge"]:
                q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], orig_obj["context"], sup[0])
                break
    output.append(q_obj)

IndexError: ignoring
Minoru Suzuki
['Minoru Suzuki (鈴木 実 , Suzuki Minoru , ring name: 鈴木 みのる) (born June 17, 1968) is a Japanese professional wrestler and mixed martial artist who is currently working for New Japan Pro Wrestling (NJPW) as a freelancer.', ' He is the current NEVER Openweight Champion in his first reign.']
[0, 1, 2]
IndexError: ignoring
Khady Sylla
['Khady Sylla (Dakar, March 27, 1963 – Dakar, October 8, 2013) was a Senegalese writer of two novels, short work, and film.']
[0, 2]
IndexError: ignoring
Harry Styles
['Harry Edward Styles (born 1 February 1994) is an English singer, songwriter, and actor.', ' He made his debut as a singer with his band White Eskimo, who performed locally in Holmes Chapel, Cheshire.', ' Styles rose to stardom as a member of the boy band One Direction, formed in 2010 through the British music competition series "The X Factor".', ' One Direction has released five albums, performed four worldwide tours, and won several awards.']
[0, 3, 4]
IndexEr

In [33]:
output[19]

{'id': '5ab698885542995eadef002a',
 'question': 'American politician Joe Heck ran unsuccessfully against Democrat Catherine Cortez Masto, a woman who previously served as the 32nd Attorney General of where?',
 'level': 'easy',
 'type': 'bridge',
 'answers': ["Joseph John “Joe” Heck (born October 30, 1961) is an American politician, physician, and U.S. Army Brigadier General who had served as the U.S. Representative for Nevada's 3rd congressional district from 2011 to 2017.",
  ' He ran unsuccessfully against Democrat Catherine Cortez Masto in the general election for the open Nevada United States Senate seat in 2016.'],
 'final_answer': 'Nevada'}

In [35]:
# Store results to disk

output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_{SPLIT}_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": output}, fp, indent=4)